Задание:

Разработать программное обеспечение, реализующее функции генерации секретного и открытого ключей, шифрования и цифровой подписи для алгоритма RSA. Обмен входными и выходными данными должен осуществляться через файлы:
1. открытого ключа;
2. секретного ключа;
3. исходного сообщения;
4. зашифрованного сообщения.
Для повышения скорости шифрования использовать метод последовательного возведения в квадрат и умножения.
Выполнить тестирование разработанного программного обеспечения на 10 наборах тестовых данных.
Длина чисел p и q должна быть не менее 1024 бит.

In [1]:
KEYSIZE = 1024

Генерация больших простых чисел:

In [2]:
import random

def rabin_miller(num):
   s = num - 1
   t = 0
   
   while s % 2 == 0:
      s = s // 2
      t += 1
   for _ in range(5):
      a = random.randrange(2, num - 1)
      v = pow(a, s, num)
      if v != 1:
         i = 0
         while v != (num - 1):
            if i == t - 1:
               return False
            else:
               i = i + 1
               v = (v ** 2) % num
      return True

def is_prime(num):
   if num < 2:
      return False
   lowPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 
   67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 
   157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 
   251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,317, 331, 337, 347, 349, 
   353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 
   457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 
   571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 
   673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 
   797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 
   911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
	
   if num in lowPrimes:
      return True
   for prime in lowPrimes:
      if (num % prime == 0):
         return False
   return rabin_miller(num)

def gcd(a, b):
   while a != 0:
      a, b = b % a, a
   return b

def generate_prime(keysize = 1024):
   while True:
      num = random.randrange(2**(keysize-1), 2**(keysize))
      if is_prime(num):
         return num
      
generate_prime()

109255353255796260226020866541162374271081182233612263070482974266011857925069193979344146191986017598926959689837754866479363419624246422940030222520377666288479303946520843478375444556609503901916566463102246631499847266477871646517925821075062224970798574537315894355721102338670235277519514117486935269677

Теперь, сгенерируем два случайных простых числа p и q, вычислим их произведение, а также функцию Эйлера. В качестве открытой экспоненты используем одно из простых чисел Ферма (17, 257, 65537).

In [3]:
p, q = generate_prime(KEYSIZE), generate_prime(KEYSIZE)

n = p * q
f = (p - 1) * (q - 1)

while True:
    e = random.randrange(2 ** (KEYSIZE - 1), 2 ** KEYSIZE)
    if gcd(e, f) == 1:
        break

Метод find_mod_inverse() вычисляет число d, мультипликативно обратное числу e по модулю n (т.е. удовлетворяет сравнению d*e = 1mod f(n))

In [4]:
def find_mod_inverse(a, m):
   if gcd(a, m) != 1:
      return None
   u1, u2, u3 = 1, 0, a
   v1, v2, v3 = 0, 1, m
   
   while v3 != 0:
      q = u3 // v3
      v1, v2, v3, u1, u2, u3 = (u1 - q * v1), (u2 - q * v2), (u3 - q * v3), v1, v2, v3
   return u1 % m

d = find_mod_inverse(e, f)

Таким образом, теперь мы можем составить закрытый и открытый ключи:

In [5]:
public_key = (e, n)
private_key = (d, n)

print(f"Закрытый ключ: {private_key}")
print(f"Открытый ключ: {public_key}")

Закрытый ключ: (1252697521871108098544353966224249283363579540484429818460644682501199759874750185653431737821463815779674619442474326501050063245353241659381347675982367132895342202531905910519254918320897010257199665365186540027584247303865402918776862324109149576018959755755769033872319790896452449743081159891594108948763148450162290198182778656291783211576445072167969020941405471876176016155619273148610042928334187516155081320568564182649699933331991596072945592780944077881492163473304470571541700374706104454534751183555393759790447529958827303575616218447628820941418869047427226650099060021903769253321988059409070317135, 196243586234410701304229159091944611632261787126519794549571550997799882786176322805241108129794350533037097041982835797709762234538590816874520892456340820621289532221958424088797796276116816947454635065545651598486556587988359656352662650968155807137106351893547238026515219023417855442333555485251194649060681682756377524246253196307562625937675294380424068010267

В качестве исходного сообщения будет выступать целое число в интервале от 0 до n - 1.

Для зашифрования используем открытый ключ {e, n}, вычилим крипограмму:

In [6]:
m = random.randint(1, n - 1)

def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m

print(f"Исходное сообщение: {m}")

c = fast_mod_exp(m, e, n)

print(f"Криптограмма: {c}")

dc = fast_mod_exp(c, d, n)

print(f"Расшифрованная криптограмма: {dc}")

Исходное сообщение: 17526795221789309735422727796879543412204972721703294145018340259006126065019635298058000921417861219374239090585797804277368608059861564757585502475686194784123406946813624159745238831999402621391308108075888093885876684353321119933081141828384752225917662488762731911391892476808026669835710018257206807763960340987483813178611919802472545094985995453654442035284449457778693185949414285078795390974326880391265138427399815591021902088629481546883939278220578604303499451888617900361794698237101653465045941346393202163053714237714158179354080734975659015873316327886703666421296789352985070110323742476204967789445
Криптограмма: 132497716252663486324336404860611445710001857518529207364310661402982882057877235766964649905126959642565087868202813904952738269508601918324121996128730456426689084580251889272471870122244478925752628495293571270997726795613701703339489980094250476147123077344642483762946890900727557679539449546912072415937798908746705982665674133634972291307929

Цифровая подпись:

Для создания цифровой подписи s с помощью секретного ключа вычисляют s = m^d mod n. Затем формируют пару {m, s} и отправляют получателю.

In [7]:
s = fast_mod_exp(m, d, n)

signature = (m, s)

print(f"{signature=}")

signature=(17526795221789309735422727796879543412204972721703294145018340259006126065019635298058000921417861219374239090585797804277368608059861564757585502475686194784123406946813624159745238831999402621391308108075888093885876684353321119933081141828384752225917662488762731911391892476808026669835710018257206807763960340987483813178611919802472545094985995453654442035284449457778693185949414285078795390974326880391265138427399815591021902088629481546883939278220578604303499451888617900361794698237101653465045941346393202163053714237714158179354080734975659015873316327886703666421296789352985070110323742476204967789445, 3070818297860095716824276025417809258546333637379112884868023349212147422536207599651393357794009725239406948902546484320376265028681184208009337103168977999157943017604275652717697518876933890700502563350305811148663386062923054250180351501750627347484193264712957065234265367232650893913790221875325974727890779762870083309844250263754163595117899621979053702171818642

Теперь проверим цифровую подпись: вычислим прообраз сообщения из подписи.

In [8]:
ms = fast_mod_exp(s, e, n)

is_valid = m == ms

print(f"{is_valid=}")

is_valid=True
